In [144]:
#import SparkSession
from pyspark.sql import SparkSession

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [145]:
#create spar session object
spark=SparkSession.builder.appName('data_processing').getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [146]:
# Load csv Dataset 
df=spark.read.csv('s3://st026320211jpcastanod/datasets/spark/Casos_positivos_de_COVID-19_en_Colombia.csv',inferSchema=True,header=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [147]:
#Columnas
df.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['ID de caso', 'Fecha de notificación', 'Codigo DIVIPOLA', 'Ciudad de ubicación', 'Departamento o Distrito ', 'atención', 'Edad', 'Sexo', 'Tipo', 'Estado', 'País de procedencia', 'FIS', 'Fecha de muerte', 'Fecha diagnostico', 'Fecha recuperado', 'fecha reporte web']

In [148]:
#Tipo de datos
df.dtypes

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[('ID de caso', 'int'), ('Fecha de notificación', 'timestamp'), ('Codigo DIVIPOLA', 'int'), ('Ciudad de ubicación', 'string'), ('Departamento o Distrito ', 'string'), ('atención', 'string'), ('Edad', 'int'), ('Sexo', 'string'), ('Tipo', 'string'), ('Estado', 'string'), ('País de procedencia', 'string'), ('FIS', 'string'), ('Fecha de muerte', 'string'), ('Fecha diagnostico', 'timestamp'), ('Fecha recuperado', 'string'), ('fecha reporte web', 'timestamp')]

In [149]:
#Seleccionar algunas columnas
df.select('edad', 'Estado').show()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+------+
|edad|Estado|
+----+------+
|  19|  Leve|
|  34|  Leve|
|  50|  Leve|
|  55|  Leve|
|  25|  Leve|
|  27|  Leve|
|  85|  Leve|
|  22|  Leve|
|  28|  Leve|
|  36|  Leve|
|  42|  Leve|
|  74|  Leve|
|  68|  Leve|
|  48|  Leve|
|  30|  Leve|
|  61|  Leve|
|  73|  Leve|
|  54|  Leve|
|  54|  Leve|
|  26|  Leve|
+----+------+
only showing top 20 rows

In [150]:
#Agregar nueva columna
df.withColumn("age_after_10_yrs",(df["edad"]+10)).show(10,False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+---------------------+---------------+-------------------+------------------------+----------+----+----+-----------+------+-------------------------+-----------------------+---------------+-------------------+-----------------------+-------------------+----------------+
|ID de caso|Fecha de notificación|Codigo DIVIPOLA|Ciudad de ubicación|Departamento o Distrito |atención  |Edad|Sexo|Tipo       |Estado|País de procedencia      |FIS                    |Fecha de muerte|Fecha diagnostico  |Fecha recuperado       |fecha reporte web  |age_after_10_yrs|
+----------+---------------------+---------------+-------------------+------------------------+----------+----+----+-----------+------+-------------------------+-----------------------+---------------+-------------------+-----------------------+-------------------+----------------+
|1         |2020-03-02 00:00:00  |11001          |Bogotá D.C.        |Bogotá D.C.             |Recuperado|19  |F   |Importado  |Leve  |Italia          

In [151]:
#Eliminar columna
df_new=df.drop('age_after_10_yrs')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [152]:
#Filtrar datos
df.filter(df['Estado']!='Leve').show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+---------------------+---------------+-------------------+------------------------+------------+----+----+-----------+---------+--------------------+--------------------+--------------------+-------------------+--------------------+-------------------+
|ID de caso|Fecha de notificación|Codigo DIVIPOLA|Ciudad de ubicación|Departamento o Distrito |    atención|Edad|Sexo|       Tipo|   Estado| País de procedencia|                 FIS|     Fecha de muerte|  Fecha diagnostico|    Fecha recuperado|  fecha reporte web|
+----------+---------------------+---------------+-------------------+------------------------+------------+----+----+-----------+---------+--------------------+--------------------+--------------------+-------------------+--------------------+-------------------+
|       152|  2020-03-18 00:00:00|          11001|        Bogotá D.C.|             Bogotá D.C.|   Fallecido|  65|   M|  Importado|Fallecido|Estados Unidos de...|2020-03-10T00:00:...|2020-04-05T00:00:...|20

In [153]:
from pyspark.sql.functions import udf

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [154]:
#función lambda
age_udf = udf(lambda edad: "young" if edad <= 30 else "senior", StringType())
#aplicando udf
df.withColumn("age_group", age_udf(df.Edad)).show(10,False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+---------------------+---------------+-------------------+------------------------+----------+----+----+-----------+------+-------------------------+-----------------------+---------------+-------------------+-----------------------+-------------------+---------+
|ID de caso|Fecha de notificación|Codigo DIVIPOLA|Ciudad de ubicación|Departamento o Distrito |atención  |Edad|Sexo|Tipo       |Estado|País de procedencia      |FIS                    |Fecha de muerte|Fecha diagnostico  |Fecha recuperado       |fecha reporte web  |age_group|
+----------+---------------------+---------------+-------------------+------------------------+----------+----+----+-----------+------+-------------------------+-----------------------+---------------+-------------------+-----------------------+-------------------+---------+
|1         |2020-03-02 00:00:00  |11001          |Bogotá D.C.        |Bogotá D.C.             |Recuperado|19  |F   |Importado  |Leve  |Italia                   |2020-02-27T

In [155]:
from pyspark.sql.functions import pandas_udf, PandasUDFType
from pyspark.sql.functions import col

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [156]:
#Departamentos con mas casos de covid
diezDepartamentosConMasCasos = df.groupBy("Departamento o Distrito ").count().sort(col("count").desc()).limit(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [157]:
diezDepartamentosConMasCasos.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------------+-----+
|Departamento o Distrito |count|
+------------------------+-----+
|             Bogotá D.C.| 2543|
|         Valle del Cauca|  877|
|               Antioquia|  468|
|                    Meta|  363|
|     Cartagena D.T. y C.|  246|
|            Cundinamarca|  219|
|               Risaralda|  193|
|       Barranquilla D.E.|  141|
|    Santa Marta D.T. ...|  139|
|                  Nariño|  118|
+------------------------+-----+

In [158]:
#Ciudades con mas casos de covid
diezCiudadesConMasCasos = maxDepartaments = df.groupBy("Ciudad de ubicación").count().sort(col("count").desc()).limit(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [159]:
diezCiudadesConMasCasos.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+-----+
|Ciudad de ubicación|count|
+-------------------+-----+
|        Bogotá D.C.| 2543|
|               Cali|  713|
|      Villavicencio|  344|
|           Medellín|  305|
|Cartagena de Indias|  246|
|       Barranquilla|  141|
|        Santa Marta|  139|
|            Pereira|  132|
|            Leticia|  104|
|              Neiva|   82|
+-------------------+-----+

In [160]:
#Dias con mas casos de covid
diezDiasConMasCasos = df.groupBy("Fecha de notificación").count().sort(col("count").desc()).limit(10)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [161]:
diezDiasConMasCasos.show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------------+-----+
|Fecha de notificación|count|
+---------------------+-----+
|  2020-04-22 00:00:00|  235|
|  2020-04-11 00:00:00|  235|
|  2020-04-16 00:00:00|  232|
|  2020-04-14 00:00:00|  207|
|  2020-04-21 00:00:00|  194|
|  2020-04-01 00:00:00|  193|
|  2020-03-30 00:00:00|  191|
|  2020-03-31 00:00:00|  188|
|  2020-04-03 00:00:00|  184|
|  2020-04-19 00:00:00|  182|
+---------------------+-----+

In [162]:
#Organizados por edades
casosPorEdad = df.groupBy("Edad").count().sort(col("count").desc())


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [163]:
casosPorEdad.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-----+
|Edad|count|
+----+-----+
|  29|  167|
|  30|  166|
|  28|  162|
|  31|  161|
|  32|  149|
|  27|  148|
|  25|  147|
|  26|  143|
|  33|  137|
|  36|  135|
|  24|  134|
|  37|  131|
|  40|  130|
|  39|  129|
|  34|  126|
|  35|  120|
|  38|  120|
|  42|  112|
|  45|  110|
|  23|  110|
+----+-----+
only showing top 20 rows

In [164]:
#Cantidad de muertos por covid por ciudad
muertosPorCiudad = df.filter(df['Estado']=='Fallecido').groupBy("Ciudad de ubicación").count().sort(col("count").desc())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [165]:
muertosPorCiudad.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----+
| Ciudad de ubicación|count|
+--------------------+-----+
|         Bogotá D.C.|   97|
|                Cali|   42|
| Cartagena de Indias|   21|
|         Santa Marta|   14|
|        Barranquilla|    6|
|             Pereira|    5|
|             Soledad|    4|
|             Palmira|    4|
|       Villavicencio|    4|
|               Neiva|    4|
|              Soacha|    4|
|              Cúcuta|    4|
|             Ipiales|    3|
|            Medellín|    3|
|          Valledupar|    3|
|           Zipaquirá|    2|
|          Río de Oro|    2|
|San Andrés de Tumaco|    2|
|         Puebloviejo|    2|
|         Villapinzón|    2|
+--------------------+-----+
only showing top 20 rows

In [166]:
write_uri='s3://st026320211jpcastanod/lab3/diesDepartamentosConMasCasos.csv'
diezDepartamentosConMasCasos.coalesce(1).write.format("csv").option("header","true").save(write_uri)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [167]:
write_uri='s3://st026320211jpcastanod/lab3/diezCiudadesConMasCasos.csv'
diezCiudadesConMasCasos.coalesce(1).write.format("csv").option("header","true").save(write_uri)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [168]:
write_uri='s3://st026320211jpcastanod/lab3/diezDiasConMasCasos.csv'
diezDiasConMasCasos.coalesce(1).write.format("csv").option("header","true").save(write_uri)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [169]:
write_uri='s3://st026320211jpcastanod/lab3/casosPorEdad.csv'
casosPorEdad.coalesce(1).write.format("csv").option("header","true").save(write_uri)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [170]:
write_uri='s3://st026320211jpcastanod/lab3/muertosPorCiudad.csv'
muertosPorCiudad.coalesce(1).write.format("csv").option("header","true").save(write_uri)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…